In [1]:
import numpy as np
import nltk, os
from word import Word

In [40]:
class n2c2Parser():    
    filepath =  None
    tokens = None
    def __init__(self, filepath):
        self.filepath = filepath

    def parse_file(self):
        with open(self.filepath, 'r') as f:
            data = f.read()
        tokens = []
        word_offset = 0
        prev_char = ' '
        current_token = ''
        start_char, end_char = None, None 
        end_of_word = {' ', '\n'} # tokens signifying end of word
        for i, char in enumerate(data):
            if prev_char in end_of_word and char not in end_of_word: # new word started
                start_char = i
                current_token += char
            elif prev_char not in end_of_word and char in end_of_word: # word ended
                end_char = i
                tokens.append(
                    {'text': current_token, 'start_char':  start_char, 'end_char':  end_char}
                )
                current_token = ''
            elif prev_char not in end_of_word and char not in end_of_word: # word continues
                current_token += char
            else: # blank space followed by blank space
                pass
            prev_char = char
        self.tokens = tokens
    
    def tag_gold_tokens(self, gold_file):
        parsed_gold = {}
        with open(gold_file, 'r') as f:
            gold_data = f.read().splitlines()
        for l in gold_data:
            data_in_line = self.parse_gold_line(l)
            for data in data_in_line:
                if data['start_line'] != None:
                    for line in range(data['start_line'], data['end_line']+1):
                        if data['start_line'] == data['end_line']:
                            for token in range(data['start_token'], data['end_token'] + 1):
                                self.tokens[line][token]['gold_tag'] = data['concept']
                                self.tokens[line][token]['gold_text'] = data['text']
                        else: # if multi-line annotation
                            if line == data['start_line']:
                                for token in self.tokens[line]:
                                    if token >= data['start_token']: 
                                        self.tokens[line][token]['gold_tag'] = data['concept']
                                        self.tokens[line][token]['gold_text'] = data['text']
                            elif line == data['end_line']:
                                for token in self.tokens[line]:
                                    if token <= data['end_token']: 
                                        self.tokens[line][token]['gold_tag'] = data['concept']
                                        self.tokens[line][token]['gold_text'] = data['text']
                            else:
                                for token in self.tokens[line]:
                                    self.tokens[line][token]['gold_tag'] = data['concept']
                                    self.tokens[line][token]['gold_text'] = data['text']

                                
    def parse_gold_line(self, line):
        bundle = []
        chunks = line.split('||')
        for chunk in chunks:
            split_a = chunk.split('\" ')
            concept = split_a[0].split('=')[0]
            text = split_a[0].split('=')[1][1:]
            
            split_b = split_a[1].split() if len(split_a) > 1 else []
            if len(split_b) == 0: # no start/end values here
                start_line, start_token = None, None
                end_line, end_token = None, None
                bundle.append({
                    'concept': concept,
                    'text': text,
                    'start_line': start_line,
                    'start_token': start_token,
                    'end_line': end_line,
                    'end_token': end_token,
                })

            elif r',' not in split_b[1]:
                [start_line, start_token] = split_b[0].split(':')
                [end_line, end_token] = split_b[1].split(':')
                bundle.append({
                    'concept': concept,
                    'text': text,
                    'start_line': int(start_line),
                    'start_token': int(start_token),
                    'end_line': int(end_line),
                    'end_token': int(end_token),
                })
            else:
                groups = split_a[1].split(r',')
                for group in groups:
                    [start, end] = group.split()
                    [start_line, start_token] = start.split(':')
                    [end_line, end_token] = end.split(':')
                    bundle.append({
                        'concept': concept,
                        'text': text,
                        'start_line': int(start_line),
                        'start_token': int(start_token),
                        'end_line': int(end_line),
                        'end_token': int(end_token),
                    })
        return bundle

# Testing

In [15]:
input_file = 100039
data_path = '/Users/samrawal/Documents/workspace/colab/data/n2c2/track2/training_20180910/'

txt_file_path = '{0}/{1}.txt'.format(data_path, input_file)
ann_file_path = '{0}/{1}.ann'.format(data_path, input_file)


In [42]:
parser = n2c2Parser(txt_file_path)
parser.parse_file()
tokens = parser.tokens

In [48]:
for token in tokens:
    if token['start_char'] == 166:
        print(token)

{'text': 'Prochlorperazine', 'start_char': 166, 'end_char': 182}
